# Dataset of Biels WSI Annotated for Gray Matter

In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
# Imports
import sys
sys.path.append('../..')

from os.path import join, isfile, splitext, basename
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact, IntSlider
from glob import glob

from neurotk import login, imwrite, imread
from neurotk.girder_utils import get_items, get_thumbnail_with_mask
from neurotk.utils import create_dirs

In [3]:
# Authenticate girder client.
gc = login('https://megabrain.neurology.emory.edu/api/v1',
           username='jvizcar', password='W00lyW00ly!')

In [4]:
# Create directories to save images and masks.
root_dir = '/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/' + \
           'gray-matter-dataset'

img_dir = join(root_dir, 'images')
mask_dir = join(root_dir, 'masks')

create_dirs((img_dir, mask_dir))

In [5]:
# Get list of IDs with annotation document "ManualGrayMatter"
items = []
                                # Emory ADRC collection
for item in tqdm(get_items(gc, '641ba814867536bb7a225533')):
    # Only look at Biels
    meta = item['meta'] if 'meta' in item else {}
    schema = meta['npSchema'] if 'npSchema' in meta else {}

    if 'stainID' not in schema or schema['stainID'] != 'Biels':
        continue
    
    # Check for the annotation document.
    docs = gc.get(
        f"annotation?itemId={item['_id']}&sort=lowerName&sortdir=1"
    )

    for doc in gc.get(
        f"annotation?itemId={item['_id']}&sort=lowerName&sortdir=1"
    ):
        if doc['annotation']['name'] == 'ManualGrayMatter':
            items.append(item)
            break

print(f'There are {len(items)} biels WSIs with gray matter annotations.')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 44875/44875 [07:34<00:00, 98.71it/s]


There are 232 biels WSIs with gray matter annotations.


In [9]:
# Loop through each WSI and save the thumbnail and binary mask.
size = 256  # size to save images / mask for training (will be square)
annotation_docs = ['ManualGrayMatter']
annotation_groups = ['Gray Matter']

print(f'Saving thumbnail images and masks as size {size}.')
for item in tqdm(items):
    # Skip some WSIs that are currently problematic.
    if item['_id'] in (
        '641ba8e0867536bb7a225865', '641ba952867536bb7a225a4b',
        '641baaa9867536bb7a225ff5', '641babbf867536bb7a22643f'
    ):
        continue

    fn = splitext(item['name'])[0] + '.png'

    img_fp = join(img_dir, fn)
    mask_fp = join(mask_dir, fn)

    if isfile(img_fp) and isfile(mask_fp):
        continue
        
    thumbnail, mask = get_thumbnail_with_mask(
        gc, item['_id'], annotation_docs=annotation_docs, 
        annotation_groups=annotation_groups, size=size                                           
    )

    imwrite(img_fp, thumbnail[:, :, :3])
    imwrite(mask_fp, mask)

Saving thumbnail images and masks as size 256.


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 232/232 [00:00<00:00, 591.81it/s]


## Interactively View Thumbnail Images with Gray Matter Masks

In [22]:
# Create interactive.
def display_thumbnail_and_gm_mask(i: int):
    """Display the thumbnail and gray matter mask.
    
    Args:
        i: Index of filepath.
        
    """
    fp = thumbnail_fps[i]
    thumbnail = imread(fp)

    mask = imread(join(mask_dir, basename(fp)))

    fig = plt.figure(figsize=(7, 4))
    fig.add_subplot(1, 2, 1)
    plt.imshow(thumbnail)
    fig.add_subplot(1, 2, 2)
    plt.imshow(mask)
    plt.axis('off')
    plt.show()


# Get list of image filepaths.
thumbnail_fps = glob('/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/' + \
                     'gray-matter-dataset/images/*.png')

_ = interact(
    display_thumbnail_and_gm_mask, 
    i=IntSlider(min=0, max=len(thumbnail_fps)-1, continuous_update=False),
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='i', max=227), Output()), _dom_c…